In [ ]:
# System & Python dependencies
!apt -y update -qq
!apt -y install -qq libgl1-mesa-glx wget git

# Clone ComfyUI
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI

# Install Python packages
!pip install -r requirements.txt

import torch
print("✅ CUDA available:", torch.cuda.is_available())
print("💡 GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


In [ ]:
# 📂 Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Define your base folder inside Google Drive
GDRIVE_BASE = '/content/drive/MyDrive/ComfyUI'

# 📁 Define required subfolders
REQUIRED_FOLDERS = [
    'models/checkpoints',
    'models/controlnet',
    'models/vae',
    'models/upscale_models',
    'models/clip',
    'models/unet',
    'custom_nodes',
    'input',
    'output',
    'temp',
    'user'
]

# ✅ Create folders if not exist
for folder in REQUIRED_FOLDERS:
    full_path = os.path.join(GDRIVE_BASE, folder)
    os.makedirs(full_path, exist_ok=True)

print("✅ All required folders are verified or created in Google Drive.")

In [ ]:
COMFYUI_PATH = '/content/ComfyUI'

LINKS = {
    'models': f'{GDRIVE_BASE}/models',
    'custom_nodes': f'{GDRIVE_BASE}/custom_nodes',
    'input': f'{GDRIVE_BASE}/input',
    'output': f'{GDRIVE_BASE}/output',
    'temp': f'{GDRIVE_BASE}/temp',
    'user': f'{GDRIVE_BASE}/user'
}

for name, target in LINKS.items():
    source = os.path.join(COMFYUI_PATH, name)
    # Remove old local folder
    if os.path.islink(source) or os.path.isdir(source):
        !rm -rf "{source}"
    # Create symlink
    os.symlink(target, source)

print("✅ ComfyUI is now fully configured to use Google Drive.")

In [ ]:
import os

# ✅ CONFIG
drive_model_path = "/content/drive/MyDrive/ComfyUI/models/checkpoints"
model_filename = "sd_xl_base_1.0.safetensors"
model_url = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors"
force_refresh = False  # 🔁 Set to True to redownload even if file exists

# ✅ Construct full path
model_path = os.path.join(drive_model_path, model_filename)

# ✅ Ensure model folder exists
os.makedirs(drive_model_path, exist_ok=True)

# ✅ Logic to download or refresh
if not os.path.exists(model_path):
    print("⬇️  Model not found in Google Drive. Downloading...")
    !wget -O "{model_path}" "{model_url}"
    print("✅ Model downloaded to Google Drive.")
elif force_refresh:
    print("🔁 Force refresh is enabled. Re-downloading model...")
    !wget -O "{model_path}" "{model_url}"
    print("✅ Model refreshed in Google Drive.")
else:
    print("✅ Model already exists in Google Drive. Skipping download.")


In [ ]:
!rm -f ngrok ngrok.zip ngrok-stable-linux-amd64.tgz
!wget -O ngrok-stable-linux-amd64.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-stable-linux-amd64.tgz
!chmod +x ngrok
!./ngrok version
!./ngrok authtoken PUT YOUR TOKEN HERE


In [ ]:
# ComfyUI Manager
!pip install "numpy==1.26.4" --force-reinstall --quiet > /dev/null 2>&1
!rm -rf /content/ComfyUI/custom_nodes/ComfyUI-Manager
!git clone --quiet https://github.com/ltdrdata/ComfyUI-Manager.git /content/ComfyUI/custom_nodes/ComfyUI-Manager
print("✅ ComfyUI-Manager installed and numpy version is set for compatibility.")


In [ ]:
import subprocess
import time
import requests

# Start ngrok tunnel to port 8188
ngrok_process = subprocess.Popen(['./ngrok', 'http', '8188'],
                                 stdout=subprocess.DEVNULL,
                                 stderr=subprocess.STDOUT)

# Wait and retry connection
for i in range(10):
    try:
        r = requests.get('http://localhost:4040/api/tunnels')
        public_url = r.json()['tunnels'][0]['public_url']
        print(f"✅ Ngrok tunnel established: {public_url}")
        break
    except Exception as e:
        print(f"⏳ Attempt {i+1}/10: Ngrok not ready yet...")
        time.sleep(2)
else:
    print("❌ Ngrok failed to start. Please restart the runtime and try again.")



In [ ]:
# Start ComfyUI with public access
%cd /content/ComfyUI
!python main.py --listen 0.0.0.0 --port 8188   --cuda-device 0
